In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.1 MB/s eta 0:00:00


# Service 1

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.metrics import mean_squared_error

# Load and preprocess data
df = pd.read_csv("Service1.csv")

features = ['latency_ms', 'memory_allocated', 'cpu_allocated', 'cpu_usage_pct', 'memory_usage_pct']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

X = df[['latency_ms', 'memory_allocated', 'cpu_allocated', 'cpu_usage_pct', 'memory_usage_pct']].values
y = df[['cpu_usage_pct', 'memory_usage_pct']].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for LSTM [batch, seq_len, input_size]
X_train = X_train.reshape(-1, 1, X.shape[1])
X_test = X_test.reshape(-1, 1, X.shape[1])

# Convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)


# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 2)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


# Optuna objective
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 32, 128)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    model = LSTMModel(input_size=5, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(10):  # Small number for tuning
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

    # Evaluation on test set
    model.eval()
    with torch.no_grad():
        preds = model(X_test_t)
        mse = mean_squared_error(y_test, preds.numpy())
    return mse


# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best trial:")
print(study.best_trial)

# Train final model
best_params = study.best_params
final_model = LSTMModel(
    input_size=5,
    hidden_size=best_params["hidden_size"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]
)
final_model.train()
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)

for epoch in range(30):
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = final_model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

# Final evaluation
final_model.eval()
with torch.no_grad():
    y_pred = final_model(X_test_t).numpy()
    mse = mean_squared_error(y_test, y_pred)
    print(f"Final Test MSE: {mse:.4f}")


[I 2025-05-05 12:48:56,761] A new study created in memory with name: no-name-9851ac7e-3cd7-474d-b5fc-6afc92f94fda
<ipython-input-2-c4cd4eedfe1b>:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
[I 2025-05-05 12:49:12,088] Trial 0 finished with value: 4.4632819563094604e-05 and parameters: {'hidden_size': 104, 'num_layers': 2, 'lr': 0.0025910103807583137, 'dropout': 0.22074883140519125, 'batch_size': 64}. Best is trial 0 with value: 4.4632819563094604e-05.
<ipython-input-2-c4cd4eedfe1b>:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
[I 2025-05-05 12:49:40,679] Trial 1 fini

Best trial:
FrozenTrial(number=18, state=1, values=[1.0182415809127999e-06], datetime_start=datetime.datetime(2025, 5, 5, 12, 53, 43, 247670), datetime_complete=datetime.datetime(2025, 5, 5, 12, 53, 51, 937712), params={'hidden_size': 100, 'num_layers': 1, 'lr': 0.004763640730565888, 'dropout': 0.16362393996886257, 'batch_size': 32}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_size': IntDistribution(high=128, log=False, low=32, step=1), 'num_layers': IntDistribution(high=3, log=False, low=1, step=1), 'lr': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64))}, trial_id=18, value=None)
Final Test MSE: 0.0000
